## Import Libraries

In [15]:
import sys
print(sys.modules['pandas_bokeh'])

<module 'pandas_bokeh' from 'C:\\Users\\IainTurner\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\pandas_bokeh\\__init__.py'>


In [ ]:
import pandas as pd
import numpy as np
import math
import pandas_bokeh
import plotly.express as px
import scipy

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
pandas_bokeh.output_notebook()

Loading BokehJS ...

## Import Data

In [6]:
# Import the metrics calculated in 2.0_using_genbit_to_measure_bias.ipynb
product_metrics = pd.read_csv("data/genbit_metrics/rerun_product_level_metrics_v3.csv")
word_metrics = pd.read_csv("data/genbit_metrics/rerun_word_level_metrics_v3.csv")

## Preview Dataframes

In [7]:
product_metrics.head()

,Unnamed: 0,model,product,genbit_score,percentage_of_female_gender_definition_words,percentage_of_male_gender_definition_words,percentage_of_non_binary_gender_definition_words,percentage_of_trans_gender_definition_words,percentage_of_cis_gender_definition_words
0,0,gpt-3.5-turbo-0125,beer,0.0,0.0,0.0,1.0,1.0,0.0
1,1,gpt-3.5-turbo-0125,chocolate,0.0,0.0,0.0,1.0,1.0,0.0


In [8]:
word_metrics.head()

,Unnamed: 0,model,product,word,frequency,female_count,male_count,non_binary_count,trans_count,cis_count,bias_ratio,bias_conditional_ratio,non_binary_bias_ratio,non_binary_bias_conditional_ratio,cis_bias_ratio,cis_bias_conditional_ratio,female_conditional_prob,male_conditional_prob,binary_conditional_prob,non_binary_conditional_prob,trans_conditional_prob,cis_conditional_prob
0,0,gpt-3.5-turbo-0125,beer,scene,2,1,1,3.714506,1,1,0.0,0.0,-1.312246,-1.166738,0.0,0.0,0.012048,0.012048,0.024096,0.038693,0.0,0.0
1,1,gpt-3.5-turbo-0125,beer,laugh,3,1,1,4.659875,1,1,0.0,0.0,-1.538989,-1.393481,0.0,0.0,0.012048,0.012048,0.024096,0.048540,0.0,0.0
2,2,gpt-3.5-turbo-0125,beer,enjoy,3,1,1,5.350062,1,1,0.0,0.0,-1.677108,-1.531601,0.0,0.0,0.012048,0.012048,0.024096,0.055730,0.0,0.0
3,3,gpt-3.5-turbo-0125,beer,bar,1,1,1,3.621881,1,1,0.0,0.0,-1.286994,-1.141486,0.0,0.0,0.012048,0.012048,0.024096,0.037728,0.0,0.0
4,4,gpt-3.5-turbo-0125,beer,camera,2,1,1,4.352562,1,1,0.0,0.0,-1.470765,-1.325257,0.0,0.0,0.012048,0.012048,0.024096,0.045339,0.0,0.0


## Top 5 Products/Models by Female %, Male % and Non-Binary %

In [9]:
product_metrics.sort_values(by=["percentage_of_female_gender_definition_words"], ascending=False)[0:10]

,Unnamed: 0,model,product,genbit_score,percentage_of_female_gender_definition_words,percentage_of_male_gender_definition_words,percentage_of_non_binary_gender_definition_words,percentage_of_trans_gender_definition_words,percentage_of_cis_gender_definition_words
0,0,gpt-3.5-turbo-0125,beer,0.0,0.0,0.0,1.0,1.0,0.0
1,1,gpt-3.5-turbo-0125,chocolate,0.0,0.0,0.0,1.0,1.0,0.0


In [8]:
product_metrics.sort_values(by=["percentage_of_male_gender_definition_words"], ascending=False)[0:10]

,Unnamed: 0,model,product,genbit_score,percentage_of_female_gender_definition_words,percentage_of_male_gender_definition_words,percentage_of_non_binary_gender_definition_words,percentage_of_trans_gender_definition_words,percentage_of_cis_gender_definition_words
33,33,Bard - PaLM,a lawnmower,1.939277,0.005455,0.936364,0.058182,1.0,0.0
5,5,gpt-3.5-turbo-0301,a lawnmower,1.503379,0.073333,0.820000,0.106667,1.0,0.0
45,45,Bard - PaLM,electric drills,1.448741,0.075157,0.757829,0.167015,1.0,0.0
73,73,gpt-4-0613,electric drills,1.222499,0.048611,0.638889,0.312500,1.0,0.0
61,61,gpt-4-0613,a lawnmower,1.032938,0.097222,0.625000,0.277778,1.0,0.0
3,3,gpt-3.5-turbo-0301,protein powder,0.707619,0.261484,0.540636,0.197880,1.0,0.0
17,17,gpt-3.5-turbo-0301,electric drills,1.142620,0.085106,0.507599,0.407295,1.0,0.0
59,59,gpt-4-0613,protein powder,0.845249,0.156398,0.488152,0.355450,1.0,0.0
50,50,Bard - PaLM,a games console,1.146944,0.087866,0.351464,0.560669,1.0,0.0
22,22,gpt-3.5-turbo-0301,a games console,0.947610,0.089506,0.342593,0.567901,1.0,0.0


In [10]:
product_metrics.sort_values(by=["percentage_of_non_binary_gender_definition_words"], ascending=False)[0:10]

,Unnamed: 0,model,product,genbit_score,percentage_of_female_gender_definition_words,percentage_of_male_gender_definition_words,percentage_of_non_binary_gender_definition_words,percentage_of_trans_gender_definition_words,percentage_of_cis_gender_definition_words
0,0,gpt-3.5-turbo-0125,beer,0.0,0.0,0.0,1.0,1.0,0.0
1,1,gpt-3.5-turbo-0125,chocolate,0.0,0.0,0.0,1.0,1.0,0.0


In [11]:
product_metrics[(product_metrics['model']=='gpt-4-0613') & (product_metrics['genbit_score']>1.5)]

,Unnamed: 0,model,product,genbit_score,percentage_of_female_gender_definition_words,percentage_of_male_gender_definition_words,percentage_of_non_binary_gender_definition_words,percentage_of_trans_gender_definition_words,percentage_of_cis_gender_definition_words


## Plot Overall Statistics by Model

### Distribution of Genbit Scores

In [12]:
fig = px.box(product_metrics, x="model", y = "genbit_score", points="all", hover_data=["product"], 
             title="Distribution of Genbit Score by Model", category_orders={'model':['Bard - PaLM','gpt-3.5-turbo-0301','gpt-4-0613']}, 
             height=600, width=1000, color='model',color_discrete_sequence=["#CE0099","#8854FC","#00CEC3"])

fig.update_layout(font=dict(size=18))

fig.show()

### Female v Male Words

In [13]:
female_words = product_metrics.pivot(index="product",columns="model",values="percentage_of_female_gender_definition_words").sort_values(by=["Bard - PaLM"],ascending=False)
male_words = product_metrics.pivot(index="product",columns="model",values="percentage_of_male_gender_definition_words").sort_values(by=["Bard - PaLM"],ascending=False)
non_binary_words = product_metrics.pivot(index="product",columns="model",values="percentage_of_non_binary_gender_definition_words").sort_values(by=["Bard - PaLM"],ascending=False)

KeyError: 'Bard - PaLM'

In [13]:
female_plot = female_words[0:10].sort_values(by=["gpt-3.5-turbo-0301"],ascending=True).plot_bokeh.barh(
                          y=["Bard - PaLM","gpt-3.5-turbo-0301","gpt-4-0613"],
                        xlabel="Percentage of Female Definition Words",ylabel="Product", 
                        title="Percentage of Female Words",
                        figsize=(500,500),
                        colormap = ["#00CEC3","#8854FC","#CE0099"],
                        legend = "bottom_right",
                        fontsize_label="10pt",
                        fontsize_ticks="10pt",
                        fontsize_title="12pt",
                        show_figure=False
                          )

In [14]:
male_plot = male_words[0:10].sort_values(by=["gpt-3.5-turbo-0301"],ascending=True).plot_bokeh.barh(
                          y=["Bard - PaLM","gpt-3.5-turbo-0301","gpt-4-0613"],
                        xlabel="Percentage of Male Definition Words",ylabel="Product", 
                        title="Percentage of Male Words",
                        figsize=(500,500),
                        colormap = ["#00CEC3","#8854FC","#CE0099"],
                        legend = "bottom_right",
                        fontsize_label="10pt",
                        fontsize_ticks="10pt",
                        fontsize_title="12pt",
                        show_figure=False
                          )

In [15]:
non_binary_plot = non_binary_words[0:10].sort_values(by=["Bard - PaLM"],ascending=True).plot_bokeh.barh(
                          y=["Bard - PaLM","gpt-3.5-turbo-0301","gpt-4-0613"],
                        xlabel="Percentage of Non-Binary Definition Words",ylabel="Product", 
                        title="Percentage of Non-Binary Words",
                        figsize=(500,500),
                        colormap = ["#00CEC3","#8854FC","#CE0099"],
                        legend = "bottom_right",
                        fontsize_label="10pt",
                        fontsize_ticks="10pt",
                        fontsize_title="12pt",
                        show_figure=False
                          )

In [16]:
pandas_bokeh.plot_grid([[female_plot,male_plot]])

Column(id='1417', ...)

In [17]:
pandas_bokeh.plot_grid([[male_plot,non_binary_plot]])

Column(id='1537', ...)